In [12]:
import numpy as np
from fastai import *
from fastai.vision import *
import torch
import os
from torchvision import transforms

**data from https://www.kaggle.com/mloey1/ahcd1**

In [13]:
data_path = Path('data')

In [14]:
test_x = np.genfromtxt(data_path/'csvTestImages 3360x1024.csv', delimiter=',')
test_y = np.genfromtxt(data_path/'csvTestLabel 3360x1.csv', delimiter=',')
test_y = test_y.astype(int) - 1
test_x = test_x.astype(int)


test_x.shape, test_y.shape, test_x[0].shape, test_y.max(), test_y.min(), max(test_x.max(axis=1))

((3360, 1024), (3360,), (1024,), 27, 0, 255)

#### Custom Dataset

In [15]:
class ArabicMnist(Dataset):
    def __init__(self, X, y=None):
        super().__init__()
        self.classes = np.unique(y)
        self.c = len(np.unique(y))
        self.X = X
        if y is not None: self.y = y

    def __getitem__(self, i):
        image = self.X[i]
        arr3D = np.tile(image[..., None], 3)
        arr3D = np.transpose(arr3D, (2, 1, 0))
        new_tensor = torch.from_numpy(arr3D).float()
        return (new_tensor, self.y[i])
    
    def __len__(self): return len(self.X)

In [16]:
test_x_reshaped = test_x.reshape([-1, 32, 32])

In [229]:
mean = test_x.mean()
std = test_x.std()
test_x = (test_x - mean) / std
test_x = (test_x + 1) / 2 * 255 # moves it between 0 and 255
test_x = np.clip(test_x, 0, 255).astype(np.uint8)
test_x_reshaped = test_x.reshape([-1, 32, 32])
# # Numpy to Torch Tensor
# X_train = torch.from_numpy(np.float32(X_train))
# y_train = torch.from_numpy(y_train.astype(np.long))
# X_valid = torch.from_numpy(np.float32(X_valid))
# y_valid = torch.from_numpy(y_valid.astype(np.long))

# train = torch.utils.data.TensorDataset(X_train, y_train)
# valid = torch.utils.data.TensorDataset(X_valid, y_valid)

In [230]:
test_x_reshaped.shape, test_x_reshaped[0]

((3360, 32, 32), array([[92, 92, 92, 92, ..., 92, 92, 92, 92],
        [92, 92, 92, 92, ..., 92, 92, 92, 92],
        [92, 92, 92, 92, ..., 92, 92, 92, 92],
        [92, 92, 92, 92, ..., 92, 92, 92, 92],
        ...,
        [92, 92, 92, 92, ..., 92, 92, 92, 92],
        [92, 92, 92, 92, ..., 92, 92, 92, 92],
        [92, 92, 92, 92, ..., 92, 92, 92, 92],
        [92, 92, 92, 92, ..., 92, 92, 92, 92]], dtype=uint8))

In [231]:
AM = ArabicMnist(test_x_reshaped[:2720], test_y[:2720])
AMV = ArabicMnist(test_x_reshaped[2720:], test_y[2720:])

#### TensorDataset

In [8]:
mean = test_x.mean()
std = test_x.std()
X_train = (test_x - mean)/std

# Numpy to Torch Tensor
X_train = torch.from_numpy(np.float32(X_train))
y_train = torch.from_numpy(test_y.astype(np.long))

X_train, y_train = X_train[:3000], y_train[:3000]
X_valid, y_valid = X_train[3000:], y_train[3000:]

train = torch.utils.data.TensorDataset(X_train, y_train)
valid = torch.utils.data.TensorDataset(X_valid, y_valid)

data = DataBunch.create(train_ds = train, valid_ds=valid)

In [9]:
# data = DataBunch.create(AM, AMV, bs=8)

In [10]:
data.c

AttributeError: 'TensorDataset' object has no attribute 'c'

In [234]:
# img1 = data.train_ds[0]
# img1

In [235]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [236]:
# %%debug
learn.fit(3, 1e-9)

Total time: 00:30
epoch  train_loss  valid_loss  error_rate
1      -0.033194   -0.060467   0.965625    (00:10)
2      0.045981    -0.042905   0.964063    (00:10)
3      -0.035805   -0.015229   0.957812    (00:09)



In [13]:
class ArabicMnist(Dataset):
    def __init__(self, X, y=None):
        super().__init__()
        self.classes = np.unique(y)
        self.c = len(np.unique(y))
        self.X = X
        if y is not None: self.y = y

    def __getitem__(self, i):
        image = self.X[i]
        arr = (image - image.mean()) / image.std() # normalizes between -1 and +1
        arr = (arr + 1) / 2 * 255 # moves it between 0 and 255
#         arr /= 255.0  ## WITH RANGE [0, 255]
#         arr = np.clip(arr, 0, 255).astype(np.uint8),
        arr3D = np.tile(arr[..., None], 3)
#         arr3D = 256 - arr3D
        arr3D = np.transpose(arr3D, (2, 1, 0))
        new_tensor = torch.from_numpy(arr3D).float()
        return (new_tensor, self.y[i])
    
    def __len__(self): return len(self.X)

In [14]:
AM = ArabicMnist(test_x_reshaped[:2720], test_y[:2720])
AMV = ArabicMnist(test_x_reshaped[2720:], test_y[2720:])

In [15]:
data = DataBunch.create(AM, AMV, bs=8)

In [16]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [17]:
# %%debug
learn.fit(3, 1e-9)

Total time: 00:29
epoch  train_loss  valid_loss  error_rate
1      0.005595    0.063722    0.970312    (00:09)
2      0.052280    -0.058503   0.956250    (00:09)
3      -0.036031   -0.035565   0.960938    (00:09)

